# Introducing Databases and SQL

## Overview

Teaching: 60  
Exercises: 5  

### Questions

- "What is a relational database and why should I use it?"
- "What is SQL?"

### Objectives

- "Describe why relational databases are useful."
- "Create and populate a database from a text file."
- "Define SQLite data types."

### Key points

- "SQL allows us to select and group subsets of data, do math and other calculations, and combine data."
- "A relational database is made up of tables which are related to each other by shared keys."
- "Different database management systems (DBMS) use slightly different vocabulary, but they are all based on the same ideas."

# Motivation

To start, let's orient ourselves in our project workflow. You are probably used to using Excel or OpenRefine to go from messy, human created data to cleaned, computer-readable data. In this lesson you're going to learn how to use a database to read in this data, and then use it for analysis and visualization.

## What is SQL?

SQL stands for Structured Query Language. SQL allows us to interact with relational databases through queries.
These queries can allow you to perform a number of actions such as: insert, select, update and delete information in a database.

## Setup: Copy the original dataset to a folder `notebooks/data`

In [ ]:
! mkdir -p data; cp -R ../files/original-data/* ./data

## Dataset Description

The data we will be using is a time-series for a small mammal community in
southern Arizona. This is part of a project studying the effects of rodents and
ants on the plant community that has been running for almost 40 years.  The
rodents are sampled on a series of 24 plots, with different experimental
manipulations controlling which rodents are allowed to access which plots.

This is a real dataset that has been used in over 100 publications. We've
simplified it for the workshop, but you can download the
[full dataset](http://esapubs.org/archive/ecol/E090/118/) and work with it using
exactly the same tools we'll learn about today.

## Questions

Let's look at some of the cleaned spreadsheets needed to complete this challenge. See the following three files in the `data` folder:

* [`data/surveys.csv`](./data/surveys.csv)
* [`data/species.csv`](./data/species.csv)
* [`data/plots.csv`](./data/plots.csv)

## Challenge

> Open each of these csv files and explore them.
> What information is contained in each file?  Specifically, if I had
> the following research questions:
>
> * How has the hindfoot length and weight of *Dipodomys* species changed over time?
> * What is the average weight of each species, per year?
> * What information can I learn about *Dipodomys* species in the 2000s, over time?
>
> What would I need to answer these questions?  Which files have the data I need? What
> operations would I need to perform if I were doing these analyses by hand?

## Goals

In order to answer the questions described above, we'll need to do the
following basic data operations:

* select subsets of the data (rows and columns)
* group subsets of data
* do math and other calculations
* combine data across spreadsheets

In addition, we don't want to do this manually!  Instead of searching
for the right pieces of data ourselves, or clicking between spreadsheets,
or manually sorting columns, we want to make the computer do the work.

In particular, we want to use a tool where it's easy to repeat our analysis
in case our data changes. We also want to do all this searching without
actually modifying our source data.

Putting our data into a relational database and using SQL will help us achieve these goals.

## Definition: *Relational Database*

> A relational database stores data in *relations* made up of *records* with *fields*.
> The relations are usually represented as *tables*;
> each record is usually shown as a row, and the fields as columns.
> In most cases, each record will have a unique identifier, called a *key*,
> which is stored as one of its fields.
> Records may also contain keys that refer to records in other tables,
> which enables us to combine information from two or more sources.

# Databases

## Why use relational databases

Using a relational database serves several purposes.

* It keeps your data separate from your analysis.
    * This means there's no risk of accidentally changing data when you analyze it.
    * If we get new data we can rerun the query.
* It's fast, even for large amounts of data.
* It improves quality control of data entry (type constraints and use of forms in MS Access, Filemaker, Oracle Application Express etc.)
* The concepts of relational database querying are core to understanding how to do similar things using programming languages such as R or Python.

## Database Management Systems

There are different database management systems to work with relational databases
such as SQLite, MySQL, Potsgresql, MSSQL Server, and many more. Each of them differ
mainly based on their scalability, but they all share the same core principles of
relational databases. In this lesson, we use SQLite to introduce you to SQL and
data retrieval from a relational database.

## Relational databases

Let's look at a pre-existing database, the `portal_mammals.sqlite`
file from the Portal Project dataset that is included in this lesson.

Note: We're going to use [ipython-sql](https://github.com/catherinedevlin/ipython-sql) extension to run SQL commands inside this Jupyter notebook. We load the extension by using `%load_ext`.

In [ ]:
%load_ext sql

Now `%sql` magic is enabled, and we can connect to the database.

In [ ]:
%sql sqlite:///data/portal_mammals.sqlite

Not only can we use SQL commands to view information stored in a database, we can also use them to view information _about_ the database (metadata). Specifically we can use the `SELECT` command to see what tables exist in the database.

In [ ]:
%%sql
SELECT
    name
FROM
    sqlite_master
WHERE
    type='table'

Each item listed here corresponds to one of the `csv` files we were exploring earlier. 

To see the contents of any table we also use the `SELECT` command.

In [ ]:
%%sql
SELECT
    *
FROM
    species
LIMIT
    5

In [ ]:
%%sql
SELECT
    *
FROM
    surveys
LIMIT
    5

In [ ]:
%%sql
SELECT
    *
FROM
    plots
LIMIT
    5

This will give us a view that we're used to - a copy of the table. Hopefully this
helps to show that a database is, in some sense, just a collection of tables,
where there's some value in the tables that allows them to be connected to each
other (the "related" part of "relational database").

Besides tables a database can contain views and functions. We'll get to views later in this lesson, and functions are an advanced topic which we won't cover, except for using the SQLite built-in function called `pragma_table_info` to view the structure of a table.

In [ ]:
%%sql
SELECT
    name,
    type
FROM 
    pragma_table_info('surveys');

In [ ]:
%%sql
SELECT
    name,
    type
FROM 
    pragma_table_info('species');

We see information about the columns, which in databases are referred to as "fields," and their assigned data types.
(The rows of a database table are called *records*.) Each field contains one variety or type of data, often numbers or text. You can see in the `surveys` table that most fields contain numbers (BIGINT, or big integer, and FLOAT, or floating point numbers/decimals) while the `species` table is entirely made up of text fields.

## To summarize

* Relational databases store data in tables with fields (columns) and records
  (rows)
* Data in tables has types, and all values in a field have
  the same type ([list of data types](#Data-types))
* Queries let us look up data or make calculations based on columns

## Database Design

* Every row-column combination contains a single *atomic* value, i.e., not
   containing parts we might want to work with separately.
* One field per type of information
* No redundant information
    * Split into separate tables with one table per class of information
    * Needs an identifier in common between tables – shared column - to
       reconnect (known as a *foreign key*).

## Import

Before we get started with writing our own queries, we'll create our own
database.  We'll be creating this database from the three `csv` files
we downloaded earlier.

We'll create a new database called `data.db`.

Note: `.sqlite` and `.db` are common conventions to use for SQLite database files.

### Delete the file if it already exists

In [ ]:
! rm -f data/data.db; ls -l data/

### Create the database

In [ ]:
%sql sqlite:///data/data.db

If you look in the `data` folder you'll see a new file called `data.db`.

In [ ]:
! ls -l data/

Note: You can connect to multiple databases at the same time. We can use the `%sql` magic on its own to see what connections we have. The asterisk indicates which database connection is active.

In [ ]:
%sql

In [ ]:
%%sql
SELECT
    name
FROM
    sqlite_master
WHERE
    type='table'

We're going to create a `surveys` table using the `CREATE TABLE` command.

In [ ]:
%%sql
CREATE TABLE surveys( 
    record_id INTEGER, 
    month INTEGER, 
    day INTEGER, 
    year INTEGER, 
    plot_id INTEGER,
    species_id TEXT, 
    sex TEXT, 
    hindfoot_length REAL, 
    weight REAL 
);

Now when we check the metadata for the database we'll see that there is a table called `surveys`.

In [ ]:
%%sql
SELECT
    name
FROM
    sqlite_master
WHERE
    type='table'

When we check the metadata for the `surveys` table it should look like the table from `portal_mammals.sqlite`.

In [ ]:
%%sql
SELECT
    name,
    type
FROM 
    pragma_table_info('surveys');

This table is currently empty though.

In [ ]:
%%sql
SELECT
    *
FROM
    surveys
LIMIT
    5

Let's import the `surveys.csv` file into the `surveys` table.

It doesn't matter if you don't understand the details of what happens below, though I'll explain it briefly afterwards.

In [ ]:
! echo "SELECT COUNT(*) FROM surveys" | sqlite3 -csv -cmd ".import --skip 1 ./data/surveys.csv surveys" data/data.db

The number returned is how many records there are in the `surveys` table. Ensure this matches the number of records in `surveys.csv`.

Breaking down the above:

1. A cell which starts with an exclamation mark runs in the Unix shell environment (sometimes called 'Bash') using the Jupyter 'shell magic' extension
2. The Unix `echo` command takes a string and then 'echoes' it as output. 
3. In this case the string we 'echo' corresponds to a query which returns the number of records in the `surveys` table - we'll learn more about the `COUNT` SQL function later.
4. The Unix pipe character `|` takes the output of `echo` and sends it as input to the `sqlite3` command-line application
5. `-csv` tells the `sqlite3` application to work in 'CSV' mode
6. `-cmd` tells the `sqlite3` application to run the command following before executing any SQL commands passed in (from `echo` in this case)
7. `.import` is a SQL command used to import files
8. We use `--skip 1` argument for `.import` in order to skip the first row of the CSV file, because it contains the headings and we don't want the headings to show up as a record in our table
9. `./data/surveys.csv` specifies the location of the CSV file we're importing
10. `surveys` specifies the table to import data in to
11. The `data/data.db` parameter tells `sqlite3` which database file to use

In [ ]:
%%sql
SELECT
    *
FROM
    surveys
LIMIT
    5

## Challenge

> - Import the `plots` and `species` tables

### Solution

In [ ]:
%%sql
CREATE TABLE plots( 
    plot_id INTEGER, 
    plot_type TEXT 
);

In [ ]:
%%sql
CREATE TABLE species( 
    species_id TEXT, 
    genus TEXT, 
    species TEXT, 
    taxa TEXT 
);

In [ ]:
%%sql
SELECT
    name
FROM
    sqlite_master
WHERE
    type='table'

In [ ]:
%%sql
SELECT
    name,
    type
FROM 
    pragma_table_info('plots');

In [ ]:
%%sql
SELECT
    name,
    type
FROM 
    pragma_table_info('species');

In [ ]:
! echo "SELECT COUNT(*) FROM plots" | sqlite3 -csv -cmd ".import --skip 1 data/plots.csv plots" data/data.db

In [ ]:
! echo "SELECT COUNT(*) FROM species" | sqlite3 -csv -cmd ".import --skip 1 data/species.csv species" data/data.db

In [ ]:
%%sql
SELECT
    *
FROM
    plots
LIMIT
    5

In [ ]:
%%sql
SELECT
    *
FROM
    species
LIMIT
    5

## Adding fields to existing tables

You can use the SQL command `ALTER TABLE <table-name> ADD COLUMN <column-name> <column-details>` to add fields to a table.

For example to add a new field called `common_name` to species:

In [ ]:
%%sql
ALTER TABLE species add COLUMN common_name TEXT

In [ ]:
%%sql
SELECT 
    * 
FROM
    species
LIMIT
    5;

Likewise to remove a field:

In [ ]:
%%sql
ALTER TABLE species drop COLUMN common_name

## Data types

| Data type                          | Description                                                                                              |
|------------------------------------|:---------------------------------------------------------------------------------------------------------|
| CHARACTER(n)                       | Character string. Fixed-length n                                                                         |
| VARCHAR(n) or CHARACTER VARYING(n) | Character string. Variable length. Maximum length n                                                      |
| BINARY(n)                          | Binary string. Fixed-length n                                                                            |
| BOOLEAN                            | Stores TRUE or FALSE values                                                                              |
| VARBINARY(n) or BINARY VARYING(n)  | Binary string. Variable length. Maximum length n                                                         |
| INTEGER(p)                         | Integer numerical (no decimal).                                                                          |
| SMALLINT                           | Integer numerical (no decimal).                                                                          |
| INTEGER                            | Integer numerical (no decimal).                                                                          |
| BIGINT                             | Integer numerical (no decimal).                                                                          |
| DECIMAL(p,s)                       | Exact numerical, precision p, scale s.                                                                   |
| NUMERIC(p,s)                       | Exact numerical, precision p, scale s. (Same as DECIMAL)                                                 |
| FLOAT(p)                           | Approximate numerical, mantissa precision p. A floating number in base 10 exponential notation.          |
| REAL                               | Approximate numerical                                                                                    |
| FLOAT                              | Approximate numerical                                                                                    |
| DOUBLE PRECISION                   | Approximate numerical                                                                                    |
| DATE                               | Stores year, month, and day values                                                                       |
| TIME                               | Stores hour, minute, and second values                                                                   |
| TIMESTAMP                          | Stores year, month, day, hour, minute, and second values                                                 |
| INTERVAL                           | Composed of a number of integer fields, representing a period of time, depending on the type of interval |
| ARRAY                              | A set-length and ordered collection of elements                                                          |
| MULTISET                           | A variable-length and unordered collection of elements                                                   |
| XML                                | Stores XML data                                                                                          |

## SQL Data Type Quick Reference

Different databases offer different choices for the data type definition.

The following table shows some of the common names of data types between the various database platforms:

| Data type                                               | Access                    | SQLServer            | Oracle             | MySQL          | PostgreSQL    |
|:--------------------------------------------------------|:--------------------------|:---------------------|:-------------------|:---------------|:--------------|
| boolean                                                 | Yes/No                    | Bit                  | Byte               | N/A            | Boolean       |
| integer                                                 | Number (integer)          | Int                  | Number             | Int / Integer  | Int / Integer |
| float                                                   | Number (single)           | Float / Real         | Number             | Float          | Numeric       |
| currency                                                | Currency                  | Money                | N/A                | N/A            | Money         |
| string (fixed)                                          | N/A                       | Char                 | Char               | Char           | Char          |
| string (variable)                                       | Text (<256) / Memo (65k+) | Varchar              | Varchar2 | Varchar        | Varchar       |
| binary object	OLE Object Memo	Binary (fixed up to 8K)   | Varbinary (<8K)           | Image (<2GB)	Long | Raw	Blob          | Text	Binary | Varbinary     |

## End of episode